# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 23 2023 12:19PM,258052,20,9196033,Granules Consumer Health,Released
1,Feb 23 2023 12:13PM,258049,19,9196583,"Zymergen, Inc.",Executing
2,Feb 23 2023 12:10PM,258048,19,PNC425480A,"Person & Covey, Inc.",Released
3,Feb 23 2023 12:10PM,258048,19,PNC421641B,"Person & Covey, Inc.",Released
4,Feb 23 2023 12:03PM,258047,19,PNC415638A,"Person & Covey, Inc.",Released
5,Feb 23 2023 12:03PM,258047,19,PNC415639A,"Person & Covey, Inc.",Released
6,Feb 23 2023 12:03PM,258047,19,PNC418642A,"Person & Covey, Inc.",Released
7,Feb 23 2023 12:03PM,258047,19,PNC425452A,"Person & Covey, Inc.",Released
8,Feb 23 2023 12:03PM,258047,19,PNC415348C,"Person & Covey, Inc.",Released
9,Feb 23 2023 12:03PM,258047,19,PNC415352B,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,258046,Executing,1
29,258047,Released,7
30,258048,Released,2
31,258049,Executing,1
32,258052,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258046,NaN,1.0,NaN
258047,NaN,NaN,7.0
258048,NaN,NaN,2.0
258049,NaN,1.0,NaN
258052,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0.0,0.0,13.0
257936,0.0,23.0,0.0
257942,0.0,25.0,0.0
257943,0.0,0.0,1.0
257983,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0,0,13
257936,0,23,0
257942,0,25,0
257943,0,0,1
257983,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,0,0,13
1,257936,0,23,0
2,257942,0,25,0
3,257943,0,0,1
4,257983,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,,,13
1,257936,,23,
2,257942,,25,
3,257943,,,1
4,257983,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 23 2023 12:19PM,258052,20,Granules Consumer Health
1,Feb 23 2023 12:13PM,258049,19,"Zymergen, Inc."
2,Feb 23 2023 12:10PM,258048,19,"Person & Covey, Inc."
4,Feb 23 2023 12:03PM,258047,19,"Person & Covey, Inc."
11,Feb 23 2023 12:00PM,258046,10,ISDIN Corporation
12,Feb 23 2023 11:56AM,258044,12,Hush Hush
13,Feb 23 2023 11:40AM,258043,19,"GCH Granules USA, Inc."
14,Feb 23 2023 11:01AM,258041,10,ISDIN Corporation
15,Feb 23 2023 11:00AM,258039,10,Eywa Pharma Inc.
35,Feb 23 2023 10:21AM,258037,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 23 2023 12:19PM,258052,20,Granules Consumer Health,,,1
1,Feb 23 2023 12:13PM,258049,19,"Zymergen, Inc.",,1,
2,Feb 23 2023 12:10PM,258048,19,"Person & Covey, Inc.",,,2
3,Feb 23 2023 12:03PM,258047,19,"Person & Covey, Inc.",,,7
4,Feb 23 2023 12:00PM,258046,10,ISDIN Corporation,,1,
5,Feb 23 2023 11:56AM,258044,12,Hush Hush,,,1
6,Feb 23 2023 11:40AM,258043,19,"GCH Granules USA, Inc.",,,1
7,Feb 23 2023 11:01AM,258041,10,ISDIN Corporation,,,1
8,Feb 23 2023 11:00AM,258039,10,Eywa Pharma Inc.,,,20
9,Feb 23 2023 10:21AM,258037,10,MedStone Pharma LLC,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 12:19PM,258052,20,Granules Consumer Health,1,,
1,Feb 23 2023 12:13PM,258049,19,"Zymergen, Inc.",,1,
2,Feb 23 2023 12:10PM,258048,19,"Person & Covey, Inc.",2,,
3,Feb 23 2023 12:03PM,258047,19,"Person & Covey, Inc.",7,,
4,Feb 23 2023 12:00PM,258046,10,ISDIN Corporation,,1,
5,Feb 23 2023 11:56AM,258044,12,Hush Hush,1,,
6,Feb 23 2023 11:40AM,258043,19,"GCH Granules USA, Inc.",1,,
7,Feb 23 2023 11:01AM,258041,10,ISDIN Corporation,1,,
8,Feb 23 2023 11:00AM,258039,10,Eywa Pharma Inc.,20,,
9,Feb 23 2023 10:21AM,258037,10,MedStone Pharma LLC,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 12:19PM,258052,20,Granules Consumer Health,1,,
1,Feb 23 2023 12:13PM,258049,19,"Zymergen, Inc.",,1,
2,Feb 23 2023 12:10PM,258048,19,"Person & Covey, Inc.",2,,
3,Feb 23 2023 12:03PM,258047,19,"Person & Covey, Inc.",7,,
4,Feb 23 2023 12:00PM,258046,10,ISDIN Corporation,,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 12:19PM,258052,20,Granules Consumer Health,1.0,NaN,NaN
1,Feb 23 2023 12:13PM,258049,19,"Zymergen, Inc.",NaN,1.0,NaN
2,Feb 23 2023 12:10PM,258048,19,"Person & Covey, Inc.",2.0,NaN,NaN
3,Feb 23 2023 12:03PM,258047,19,"Person & Covey, Inc.",7.0,NaN,NaN
4,Feb 23 2023 12:00PM,258046,10,ISDIN Corporation,NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 12:19PM,258052,20,Granules Consumer Health,1.0,0.0,0.0
1,Feb 23 2023 12:13PM,258049,19,"Zymergen, Inc.",0.0,1.0,0.0
2,Feb 23 2023 12:10PM,258048,19,"Person & Covey, Inc.",2.0,0.0,0.0
3,Feb 23 2023 12:03PM,258047,19,"Person & Covey, Inc.",7.0,0.0,0.0
4,Feb 23 2023 12:00PM,258046,10,ISDIN Corporation,0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3354295,136.0,8.0,0.0
12,258044,1.0,0.0,0.0
15,1547678,18.0,48.0,0.0
16,258010,1.0,0.0,0.0
19,1290188,21.0,24.0,2.0
20,258052,1.0,0.0,0.0
21,258000,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3354295,136.0,8.0,0.0
1,12,258044,1.0,0.0,0.0
2,15,1547678,18.0,48.0,0.0
3,16,258010,1.0,0.0,0.0
4,19,1290188,21.0,24.0,2.0
5,20,258052,1.0,0.0,0.0
6,21,258000,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,136.0,8.0,0.0
1,12,1.0,0.0,0.0
2,15,18.0,48.0,0.0
3,16,1.0,0.0,0.0
4,19,21.0,24.0,2.0
5,20,1.0,0.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,136.0
1,12,Released,1.0
2,15,Released,18.0
3,16,Released,1.0
4,19,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
Executing,8.0,0.0,48.0,0.0,24.0,0.0,0.0
Released,136.0,1.0,18.0,1.0,21.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,8.0,0.0,48.0,0.0,24.0,0.0,0.0
2,Released,136.0,1.0,18.0,1.0,21.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,8.0,0.0,48.0,0.0,24.0,0.0,0.0
2,Released,136.0,1.0,18.0,1.0,21.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()